In [5]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('New_intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Shirjeel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Shirjeel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
buyer_offers_list = []


def clean_up_sentence(sentence):
    # tokenize the pattern - splitting words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stemming every word - reducing to base form
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for words that exist in sentence
def bag_of_words(sentence, words, show_details=True):
    # tokenizing patterns
    sentence_words = clean_up_sentence(sentence)
    # bag of words - vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,word in enumerate(words):
            if word == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % word)
    return(np.array(bag))

def predict_class(sentence):
    # filter below  threshold predictions
    p = bag_of_words(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sorting strength probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result


#Creating tkinter GUI
import tkinter
from tkinter import *



def new_amount_offer(buyer_offer, new_offer, actual_amount, thresh, buyer_offers_list):
    temp = 0
    iterator = len(buyer_offers_list)
    if iterator==1:
        price_cut = buyer_offer * 100/actual_amount
        temp = new_offer - price_cut
        temp = temp/10
        temp = temp * 0.10
    else:
        price_cut = buyer_offer * 100/actual_amount
        temp = new_offer - price_cut
        temp = (temp/10)
        temp = temp * 0.10 
        return (temp*100)-(iterator*10)
    
    return (temp*100)
        

def extract_amount(sentence):
    try:
        sentence = sentence.lower()
        if "rs." in sentence:
            price_start = str.find(sentence, "rs.")
            price_end = str.find(sentence, "/-")
            amount = sentence[price_start + 3:price_end]
            amount = amount.strip()  
            return float(amount)
    except:
        return 'invalid_amount'
    


def send():
    
    actual_amt = 1000
    thresh = 750
    new_offer = actual_amt 
    
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    buyer_offer = extract_amount(msg)
    concat_msg = ""
    if(buyer_offer!=None):
        
        buyer_offers_list.append(buyer_offer)
        max_offer = max(buyer_offers_list)
        buyer_offers_list.index(max_offer)
        
        if len(buyer_offers_list)>1:
            
            if buyer_offers_list[-1]<max_offer:
                concat_msg += "Are you playing? You have offered more than a few seconds ago."
                if max_offer<thresh:
                    concat_msg += "All of your offers are rejected. You are offering less than no-loss/no-profit amount."
            else:

                new_offer = new_amount_offer(buyer_offer, new_offer, actual_amt, thresh, buyer_offers_list)
                if buyer_offer<new_offer:
                    concat_msg += "New Offer:" + str(round(new_offer))
                else:
                    concat_msg += "Deal Done!"
        else:
            new_offer = new_amount_offer(buyer_offer, new_offer, actual_amt, thresh, buyer_offers_list)
            if buyer_offer<new_offer:
                concat_msg += "New Offer:" + str(round(new_offer))
            else:
                concat_msg += "Deal Done!"
    else:
        buyer_offers_list.append(0)
                

    if msg != '':
        ChatBox.config(state=NORMAL)
        ChatBox.insert(END, "You: " + msg + '\n\n')
        ChatBox.config(foreground="#446665", font=("Verdana", 12 ))
    
        ints = predict_class(msg)
        res = getResponse(ints, intents)
        if concat_msg == "Deal Done!":
            ChatBox.insert(END, "Bot: " +concat_msg +"\n\n")
        else:
            ChatBox.insert(END, "Bot: " +concat_msg +'\n'+ res + "\n\n")
            
            
        ChatBox.config(state=DISABLED)
        ChatBox.yview(END)
 

root = Tk()
root.title("B-bot")
root.geometry("400x500")
root.resizable(width=FALSE, height=FALSE)



#Create Chat window
ChatBox = Text(root, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatBox.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(root, command=ChatBox.yview, cursor="heart")
ChatBox['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(root, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#f9a602", activebackground="#3c9d9b",fg='#000000',
                    command= send )

#Create the box to enter message
EntryBox = Text(root, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatBox.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)


root.mainloop()
